In [1]:
import torch
from data_loaders.liar.liar_dataloaders import train_loader, validation_loader, test_loader
from train_functions.validate import validate_one_batch
from models.BaseLine import BiLSTMTextClassifierModel
from train_functions.train import train_one_batch
# Train model
from torch import nn
import time


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/sharaf/github/data-science-lab/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/sharaf/github/data-science-lab/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/sharaf/github/data-science-lab/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", 

In [2]:
from dataset_classes.liar.liar_dataset import train_dataset


liar_statements_vocab = train_dataset.statement_vocab

In [3]:
import os
DEVICE_NAME = os.getenv("DEVICE")
device = torch.device(DEVICE_NAME)


In [4]:
model = BiLSTMTextClassifierModel(liar_statements_vocab, 300, 128, 6)

In [ ]:

# Training params
num_epochs = 50
# Hyper parameters
learning_rate = 0.001

# Define loss and optomizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Send tensors to device
model.to(device)
loss_fn.to(device)

# performance tracking
train_loss = []
val_loss = []
val_accuracy_history = []
# training cycle start 
model.train()
for epoch in range(num_epochs):
    start_time = time.time()
    # Training
    model.train()
    total_train_loss = 0. 
    total_data_points_train = 0
    for train_batch in train_loader:
        inputs = train_batch["statement"].to(device)
        targets = train_batch["label"].to(device)
        
        train_batch_loss = train_one_batch(model, inputs, targets, optimizer, loss_fn)
        
        total_train_loss += train_batch_loss
        total_data_points_train += len(inputs)
    
    
    average_epoch_loss_train = total_train_loss/total_data_points_train
    train_loss.append(average_epoch_loss_train)
    # Validation
    # TODO restructure
    model.eval()        
    total_val_loss = 0. 
    total_data_points_val = 0
    true_positives_val = 0
    for validation_batch in validation_loader:
        inputs_val = validation_batch["statement"].to(device)
        targets_val = validation_batch["label"].to(device)
        
        # Returns loss and true positives count
        batch_loss_val, true_positives_count = validate_one_batch(model, inputs_val, targets_val, loss_fn)
            
        # calculate average loss and appends true positives count
        total_val_loss += batch_loss_val
        total_data_points_val += len(inputs_val)
        true_positives_val += true_positives_count
    
    average_epoch_loss_val = total_val_loss/total_data_points_val
    val_accuracy = true_positives_val/total_data_points_val
    val_accuracy_history.append(val_accuracy)    
    val_loss.append(average_epoch_loss_val)
    
        
    # Print every epoch's metrics
    elapsed_time = time.time() - start_time
    print(f"epoch {epoch + 1}, average train loss: {average_epoch_loss_train}, average val loss: {average_epoch_loss_val}, val accuracy: {val_accuracy},  training time : {elapsed_time}")
    
      

[1.9438041184097528]
epoch 1, average train loss: 1.9438041184097528, average val loss: 1.9531857202357592, val accuracy: 0.09034267912772585,  training time : 11.885324954986572
[1.9438041184097528, 1.9616277776658535]
epoch 2, average train loss: 1.9616277776658535, average val loss: 1.9532346376377474, val accuracy: 0.09034267912772585,  training time : 9.075904130935669


KeyboardInterrupt: 